# Imports

In [11]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import random
random.seed(10)

# Utils

In [12]:
def gsk(dict):
    return sorted(list(dict.keys()))

# Inputs

In [13]:
slots = list(range(0, 14))
campuses = list(range(0, 3))
courses = {i: {"campus": random.choice(campuses), "is_lab": random.uniform(0, 1)>0.85} for i in range(0, 25)}
students = {200010000+i: set(random.sample(gsk(courses), random.randint(4 ,9))) for i in range(0, 200)}
venues = {i: {"campus": random.choice(campuses), "capacity": random.randint(20, 40)} for i in range(0, 30)}
print("Data Created")

Data Created


# Model

In [14]:
model = gp.Model("Midsem_Scheduling")
print("Model Created")
X = {}
for slot in slots:
    for course in gsk(courses):
        for venue in gsk(venues):
            for student in gsk(students):
                X[slot, course, venue, student] = model.addVar(vtype=GRB.BINARY, name=f'X_{slot}_{course}_{venue}_{student}')
print(f"Decision Variables Created: {len(X)}")

Model Created
Decision Variables Created: 2100000


# Constraints

### 1. Each student must have exactly 1 exam for each course enrolled in, and 0 if not enrolled

In [15]:
for student in gsk(students):
    for course in gsk(courses):
        model.addConstr(quicksum(X[slot, course, venue, student] 
                                    for slot in slots
                                    for venue in gsk(venues)) == int(course in students[student]),
                        name=f'1_{student}_{course}')

### 2. Student has atmost 1 exam per slot

In [ ]:
for student in gsk(students):
    for slot in slots:
        model.addConstr(quicksum(X[slot, course, venue, student] 
                                    for course in gsk(courses)
                                    for venue in gsk(venues)) <= 1,
                        name=f'2_{student}_{day}_{slot}')

### 3. Theory courses are scheduled in a single slot